# Get it to fricking work for now

In [11]:
#%matplotlib inline

import pandas
from sklearn.cross_validation import train_test_split
import numpy as np
import time
from sklearn.externals import joblib
import Recommenders as Recommenders
import Evaluation as Evaluation

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
triplets_file = 'https://static.turi.com/datasets/millionsong/10000.txt'
songs_metadata_file = 'https://static.turi.com/datasets/millionsong/song_data.csv'

song_df_1 = pandas.read_table(triplets_file,header=None)
song_df_1.columns = ['user_id', 'song_id', 'listen_count']

song_df_2 =  pandas.read_csv(songs_metadata_file)

song_df = pandas.merge(song_df_1, song_df_2.drop_duplicates(['song_id']), on="song_id", how="left") 

In [13]:
song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999


In [14]:
song_df = song_df.head(10000)
song_df['song'] = song_df['title'].map(str) + " - " + song_df['artist_name']

In [15]:
song_grouped = song_df.groupby(['song']).agg({'listen_count': 'count'}).reset_index()
grouped_sum = song_grouped['listen_count'].sum()
song_grouped['percentage']  = song_grouped['listen_count'].div(grouped_sum)*100
song_grouped.sort_values(['listen_count', 'song'], ascending = [0,1])

,song,listen_count,percentage
3660,Sehr kosmisch - Harmonia,45,0.45
4678,Undo - Björk,32,0.32
5105,You're The One - Dwight Yoakam,32,0.32
1071,Dog Days Are Over (Radio Edit) - Florence + Th...,28,0.28
3655,Secrets - OneRepublic,28,0.28
4378,The Scientist - Coldplay,27,0.27
4712,Use Somebody - Kings Of Leon,27,0.27
3476,Revelry - Kings Of Leon,26,0.26
1387,Fireflies - Charttraxx Karaoke,24,0.24
1862,Horn Concerto No. 4 in E flat K495: II. Romanc...,23,0.23


In [16]:
users = song_df['user_id'].unique()
len(users)

365

In [17]:
songs = song_df['song'].unique()
len(songs)

5151

In [18]:
train_data, test_data = train_test_split(song_df, test_size = 0.20, random_state=0)
print(train_data.head(5))

                                       user_id             song_id  \
7389  94d5bdc37683950e90c56c9b32721edb5d347600  SOXNZOW12AB017F756   
9275  1012ecfd277b96487ed8357d02fa8326b13696a5  SOXHYVQ12AB0187949   
2995  15415fa2745b344bce958967c346f2a89f792f63  SOOSZAZ12A6D4FADF8   
5316  ffadf9297a99945c0513cd87939d91d8b602936b  SOWDJEJ12A8C1339FE   
356   5a905f000fc1ff3df7ca807d57edb608863db05d  SOAMPRJ12A8AE45F38   

      listen_count                 title  \
7389             2      Half Of My Heart   
9275             1  The Beautiful People   
2995             1     Sanctify Yourself   
5316             4     Heart Cooks Brain   
356             20                 Rorol   

                                                release      artist_name  \
7389                                     Battle Studies       John Mayer   
9275             Antichrist Superstar (Ecopac Explicit)   Marilyn Manson   
2995                             Glittering Prize 81/92     Simple Minds   
5316  Ever

In [19]:
#Recommenders.popularity_recommender_py

In [20]:
#popularity based recommender - benchmark for our RecSys
pm = Recommenders.popularity_recommender_py()
pm.create(train_data, 'user_id', 'song')

In [21]:
user_id = users[5]
pm.recommend(user_id) #predictions

,user_id,song,score,Rank
3194,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Sehr kosmisch - Harmonia,37,1.0
4083,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Undo - Björk,27,2.0
931,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Dog Days Are Over (Radio Edit) - Florence + Th...,24,3.0
4443,4bd88bfb25263a75bbdd467e74018f4ae570e5df,You're The One - Dwight Yoakam,24,4.0
3034,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Revelry - Kings Of Leon,21,5.0
3189,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Secrets - OneRepublic,21,6.0
4112,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Use Somebody - Kings Of Leon,21,7.0
1207,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Fireflies - Charttraxx Karaoke,20,8.0
1577,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Hey_ Soul Sister - Train,19,9.0
1626,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Horn Concerto No. 4 in E flat K495: II. Romanc...,19,10.0


In [22]:
user_id = users[8]
pm.recommend(user_id)


,user_id,song,score,Rank
3194,9bb911319fbc04f01755814cb5edb21df3d1a336,Sehr kosmisch - Harmonia,37,1.0
4083,9bb911319fbc04f01755814cb5edb21df3d1a336,Undo - Björk,27,2.0
931,9bb911319fbc04f01755814cb5edb21df3d1a336,Dog Days Are Over (Radio Edit) - Florence + Th...,24,3.0
4443,9bb911319fbc04f01755814cb5edb21df3d1a336,You're The One - Dwight Yoakam,24,4.0
3034,9bb911319fbc04f01755814cb5edb21df3d1a336,Revelry - Kings Of Leon,21,5.0
3189,9bb911319fbc04f01755814cb5edb21df3d1a336,Secrets - OneRepublic,21,6.0
4112,9bb911319fbc04f01755814cb5edb21df3d1a336,Use Somebody - Kings Of Leon,21,7.0
1207,9bb911319fbc04f01755814cb5edb21df3d1a336,Fireflies - Charttraxx Karaoke,20,8.0
1577,9bb911319fbc04f01755814cb5edb21df3d1a336,Hey_ Soul Sister - Train,19,9.0
1626,9bb911319fbc04f01755814cb5edb21df3d1a336,Horn Concerto No. 4 in E flat K495: II. Romanc...,19,10.0


In [23]:
#Recommenders.item_similarity_recommender_py

In [24]:
#item based similarity for second benchmark
is_model = Recommenders.item_similarity_recommender_py()
is_model.create(train_data, 'user_id', 'song')

In [25]:
#Print the songs for the user in training data
user_id = users[5]
user_items = is_model.get_user_items(user_id)
#
print("------------------------------------------------------------------------------------")
print("Training data songs for the user userid: %s:" % user_id)
print("------------------------------------------------------------------------------------")

for user_item in user_items:
    print(user_item)

print("----------------------------------------------------------------------")
print("Recommendation process going on:")
print("----------------------------------------------------------------------")

#Recommend songs for the user using personalized model
is_model.recommend(user_id)

------------------------------------------------------------------------------------
Training data songs for the user userid: 4bd88bfb25263a75bbdd467e74018f4ae570e5df:
------------------------------------------------------------------------------------
Just Lose It - Eminem
Without Me - Eminem
16 Candles - The Crests
Speechless - Lady GaGa
Push It - Salt-N-Pepa
Ghosts 'n' Stuff (Original Instrumental Mix) - Deadmau5
Say My Name - Destiny's Child
My Dad's Gone Crazy - Eminem / Hailie Jade
The Real Slim Shady - Eminem
Somebody To Love - Justin Bieber
Forgive Me - Leona Lewis
Missing You - John Waite
Ya Nada Queda - Kudai
----------------------------------------------------------------------
Recommendation process going on:
----------------------------------------------------------------------
No. of unique songs for the user: 13
no. of unique songs in the training set: 4483
Non zero values in cooccurence_matrix :2097


,user_id,song,score,rank
0,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Superman - Eminem / Dina Rae,0.088692,1
1,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Mockingbird - Eminem,0.067663,2
2,4bd88bfb25263a75bbdd467e74018f4ae570e5df,I'm Back - Eminem,0.065385,3
3,4bd88bfb25263a75bbdd467e74018f4ae570e5df,U Smile - Justin Bieber,0.064525,4
4,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Here Without You - 3 Doors Down,0.062293,5
5,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Hellbound - J-Black & Masta Ace,0.055769,6
6,4bd88bfb25263a75bbdd467e74018f4ae570e5df,The Seed (2.0) - The Roots / Cody Chestnutt,0.052564,7
7,4bd88bfb25263a75bbdd467e74018f4ae570e5df,I'm The One Who Understands (Edit Version) - War,0.052564,8
8,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Falling - Iration,0.052564,9
9,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Armed And Ready (2009 Digital Remaster) - The ...,0.052564,10


In [26]:
user_id = users[7]
user_items = is_model.get_user_items(user_id)
#
print("------------------------------------------------------------------------------------")
print("Training data songs for the user userid: %s:" % user_id)
print("------------------------------------------------------------------------------------")

for user_item in user_items:
    print(user_item)

print("----------------------------------------------------------------------")
print("Recommendation process going on:")
print("----------------------------------------------------------------------")

#Recommend songs using personalized model
is_model.recommend(user_id)


------------------------------------------------------------------------------------
Training data songs for the user userid: 9d6f0ead607ac2a6c2460e4d14fb439a146b7dec:
------------------------------------------------------------------------------------
Swallowed In The Sea - Coldplay
Life In Technicolor ii - Coldplay
Life In Technicolor - Coldplay
The Scientist - Coldplay
Trouble - Coldplay
Strawberry Swing - Coldplay
Lost! - Coldplay
Clocks - Coldplay
----------------------------------------------------------------------
Recommendation process going on:
----------------------------------------------------------------------
No. of unique songs for the user: 8
no. of unique songs in the training set: 4483
Non zero values in cooccurence_matrix :3429


,user_id,song,score,rank
0,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,She Just Likes To Fight - Four Tet,0.281579,1
1,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Warning Sign - Coldplay,0.281579,2
2,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,We Never Change - Coldplay,0.281579,3
3,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Puppetmad - Puppetmastaz,0.281579,4
4,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,God Put A Smile Upon Your Face - Coldplay,0.281579,5
5,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Susie Q - Creedence Clearwater Revival,0.281579,6
6,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,The Joker - Fatboy Slim,0.281579,7
7,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Korg Rhythm Afro - Holy Fuck,0.281579,8
8,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,This Unfolds - Four Tet,0.281579,9
9,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,high fives - Four Tet,0.281579,10


In [27]:
###THIS IS THE CLASS YOU NEED TO SEE FIRST

In [28]:
#sample similar songs - format = ['song_name' - 'artist_name']
is_model.get_similar_items(['U Smile - Justin Bieber'])

no. of unique songs in the training set: 4483
Non zero values in cooccurence_matrix :271


,user_id,song,score,rank
0,,Somebody To Love - Justin Bieber,0.428571,1
1,,Bad Company - Five Finger Death Punch,0.375000,2
2,,Love Me - Justin Bieber,0.333333,3
3,,One Time - Justin Bieber,0.333333,4
4,,Here Without You - 3 Doors Down,0.333333,5
5,,Stuck In The Moment - Justin Bieber,0.333333,6
6,,Teach Me How To Dougie - California Swag District,0.333333,7
7,,Paper Planes - M.I.A.,0.333333,8
8,,Already Gone - Kelly Clarkson,0.333333,9
9,,The Funeral (Album Version) - Band Of Horses,0.300000,10


In [30]:
#PERSONALIZED RECOMMENDATION

song = 'Yellow - Coldplay'
is_model.get_similar_items([song])

no. of unique songs in the training set: 4483
Non zero values in cooccurence_matrix :969


,user_id,song,score,rank
0,,Fix You - Coldplay,0.375000,1
1,,Creep (Explicit) - Radiohead,0.291667,2
2,,Clocks - Coldplay,0.280000,3
3,,Seven Nation Army - The White Stripes,0.250000,4
4,,Paper Planes - M.I.A.,0.208333,5
5,,Halo - Beyoncé,0.200000,6
6,,The Funeral (Album Version) - Band Of Horses,0.181818,7
7,,In My Place - Coldplay,0.181818,8
8,,Kryptonite - 3 Doors Down,0.166667,9
9,,When You Were Young - The Killers,0.166667,10


In [31]:
#Evaluation.precision_recall_calculator - p vs r calculator class as an eval measure. Take note, Ananya.

In [32]:
start = time.time()
user_sample = 0.05
pr = Evaluation.precision_recall_calculator(test_data, train_data, pm, is_model)
(pm_avg_precision_list, pm_avg_recall_list, ism_avg_precision_list, ism_avg_recall_list) = pr.calculate_measures(user_sample)

end = time.time()
print(end - start)

Length of user_test_and_training:319
Length of user sample:15
Getting recommendations for user:eb1ad31d040406c6428c5f4420b0bc709b1c5350
No. of unique songs for the user: 49
no. of unique songs in the training set: 4483
Non zero values in cooccurence_matrix :5949
Getting recommendations for user:1a39cf33853fd23d2242cf7b13cc8eb445befdd7
No. of unique songs for the user: 4
no. of unique songs in the training set: 4483
Non zero values in cooccurence_matrix :180
Getting recommendations for user:0a004c08b700e4edb74b44c2dbceca4280760a9a
No. of unique songs for the user: 11
no. of unique songs in the training set: 4483
Non zero values in cooccurence_matrix :760
Getting recommendations for user:2341121b6d6d2020303f02053dad60586e41034d
No. of unique songs for the user: 8
no. of unique songs in the training set: 4483
Non zero values in cooccurence_matrix :2121
Getting recommendations for user:1b704d4cddabea8258bd93497fcb73eab32fa592
No. of unique songs for the user: 117
no. of unique songs in the

In [33]:
#DONT RUN - Code only for visualization

"""import pylab as pl

#Method to generate precision and recall curve
def plot_precision_recall(m1_precision_list, m1_recall_list, m1_label, m2_precision_list, m2_recall_list, m2_label):
    pl.clf()    
    pl.plot(m1_recall_list, m1_precision_list, label=m1_label)
    pl.plot(m2_recall_list, m2_precision_list, label=m2_label)
    pl.xlabel('Recall')
    pl.ylabel('Precision')
    pl.ylim([0.0, 0.20])
    pl.xlim([0.0, 0.20])
    pl.title('Precision-Recall curve')
    #pl.legend(loc="upper right")
    pl.legend(loc=9, bbox_to_anchor=(0.5, -0.2))
    pl.show()
"""

'import pylab as pl\n\n#Method to generate precision and recall curve\ndef plot_precision_recall(m1_precision_list, m1_recall_list, m1_label, m2_precision_list, m2_recall_list, m2_label):\n    pl.clf()    \n    pl.plot(m1_recall_list, m1_precision_list, label=m1_label)\n    pl.plot(m2_recall_list, m2_precision_list, label=m2_label)\n    pl.xlabel(\'Recall\')\n    pl.ylabel(\'Precision\')\n    pl.ylim([0.0, 0.20])\n    pl.xlim([0.0, 0.20])\n    pl.title(\'Precision-Recall curve\')\n    #pl.legend(loc="upper right")\n    pl.legend(loc=9, bbox_to_anchor=(0.5, -0.2))\n    pl.show()\n'

In [34]:
"""print("Plotting precision recall curves.")

plot_precision_recall(pm_avg_precision_list, pm_avg_recall_list, "popularity_model",
                      ism_avg_precision_list, ism_avg_recall_list, "item_similarity_model")
"""

'print("Plotting precision recall curves.")\n\nplot_precision_recall(pm_avg_precision_list, pm_avg_recall_list, "popularity_model",\n                      ism_avg_precision_list, ism_avg_recall_list, "item_similarity_model")\n'

In [35]:
#generate p vs r curve in pickled results on larger datasubsets. note that we used only 10k in the beginning, we are not expanding to 100k or more
"""print("Plotting precision recall curves for a larger subset of data (100,000 rows) (user sample = 0.005).")

pm_avg_precision_list = joblib.load('pm_avg_precision_list_2.pkl')
pm_avg_recall_list = joblib.load('pm_avg_recall_list_2.pkl')
ism_avg_precision_list = joblib.load('ism_avg_precision_list_2.pkl')
ism_avg_recall_list = joblib.load('ism_avg_recall_list_2.pkl')

print("Plotting precision recall curves.")
plot_precision_recall(pm_avg_precision_list, pm_avg_recall_list, "popularity_model",
                      ism_avg_precision_list, ism_avg_recall_list, "item_similarity_model")"""

'print("Plotting precision recall curves for a larger subset of data (100,000 rows) (user sample = 0.005).")\n\npm_avg_precision_list = joblib.load(\'pm_avg_precision_list_2.pkl\')\npm_avg_recall_list = joblib.load(\'pm_avg_recall_list_2.pkl\')\nism_avg_precision_list = joblib.load(\'ism_avg_precision_list_2.pkl\')\nism_avg_recall_list = joblib.load(\'ism_avg_recall_list_2.pkl\')\n\nprint("Plotting precision recall curves.")\nplot_precision_recall(pm_avg_precision_list, pm_avg_recall_list, "popularity_model",\n                      ism_avg_precision_list, ism_avg_recall_list, "item_similarity_model")'

In [36]:
#use a matrix factorization for RecSys instead of word embeddings. Ananya said it was easier to do, so there it is.   ._.
#SVD instead of NNMF to reduce time complexity

import math as mt
import csv
from sparsesvd import sparsesvd #used for matrix factorization
import numpy as np
from scipy.sparse import csc_matrix #used for sparse matrix
from scipy.sparse.linalg import * #used for matrix multiplication

#Note: You may need to install the library sparsesvd. 
#Documentation for the sparsesvd method can be found at https://pypi.python.org/pypi/sparsesvd/

In [37]:
#constants for User Rating Matrix - (URM)
#SVD and recommendations

MAX_PID, MAX_UID = 4,5

def computeSVD(urm, K):
    U, s, Vt = sparsesvd(urm, K)

    dim = (len(s), len(s))
    S = np.zeros(dim, dtype=np.float32)
    for i in range(0, len(s)):
        S[i,i] = mt.sqrt(s[i])

    U = csc_matrix(np.transpose(U), dtype=np.float32)
    S = csc_matrix(S, dtype=np.float32)
    Vt = csc_matrix(Vt, dtype=np.float32)
    
    return U, S, Vt

def computeEstimatedRatings(urm, U, S, Vt, uTest, K, test):
    rightTerm = S*Vt 

    estimatedRatings = np.zeros(shape=(MAX_UID, MAX_PID), dtype=np.float16)
    for userTest in uTest:
        prod = U[userTest, :]*rightTerm
        estimatedRatings[userTest, :] = prod.todense()
        recom = (-estimatedRatings[userTest, :]).argsort()[:250]
    return recom


In [51]:
def compute_rating():
    Songs_rated_for=[]
    list_of_rated_songs=[]
    print "Enter songs you want user to rate : "
    for i in range(4):
        Songs_rated_for.append(raw_input())
    #print Songs_rated_for
    for i in range(5):
        print "\nEnter User ",i,"'s rating for the following songs : \n"
        temp=[]
        for j in range(len(Songs_rated_for)):
            print "Song : ",Songs_rated_for[j]
            temp.append(int(raw_input("Enter Rating given : ")))
        list_of_rated_songs.append(temp)
    return list_of_rated_songs, Songs_rated_for

In [52]:
#Used in SVD calculation (number of latent factors) for predictions on a test user
#THIS IS ALSO NEEDED TO BE SHOWN TO THE USER, SO FACTOR IT IN, BOYS.
K=2

User_rating_for_songs, Song_list_for_rec= [],[]
#u1_rating,u2_rating,u3_rating,u4_rating,u5_rating=[],[],[],[],[]
User_rating_for_songs, Song_list_for_rec= compute_rating()
#print User_rating_for_songs,"\n Songs : ",Song_list_for_rec
urm = np.array(User_rating_for_songs)
urm = csc_matrix(urm, dtype=np.float32)

U, S, Vt = computeSVD(urm, K)

uTest = [4]
print("User id for whom recommendations are needed: %d" % uTest[0])
#print("Recommended songs and ratings:")
uTest_recommended_items = computeEstimatedRatings(urm, U, S, Vt, uTest, K, True)
#for i in range(len(uTest_recommended_items)):
    #print Song_list_for_rec[i]," : ",uTest_recommended_items[i]   
lets_recommend,maxrec,minrec=0, -1,float('inf')
for i in range(len(uTest_recommended_items)):
    if uTest_recommended_items[i]>maxrec:
        maxrec=uTest_recommended_items[i]
        
    if uTest_recommended_items[i]<minrec:
        minrec=uTest_recommended_items[i]       
#for i in range(len(uTest_recommended_items)):
    #print 0.25+(maxrec/minrec),0.25+uTest_recommended_items[i]
    #if (0.25+uTest_recommended_items[i]) >= 0.25+(maxrec/minrec):
        #print "\nRecommended Song : ",Song_list_for_rec[i]
for i in range(len(uTest_recommended_items)):
    if uTest_recommended_items[i] == 0 or uTest_recommended_items[i] == 1:
        print "\nRecommended Song : ",Song_list_for_rec[i]

#print(uTest_recommended_items)

Enter songs you want user to rate : 
love
young and dumb
apocalypse
opera house

Enter User  0 's rating for the following songs : 

Song :  love
Enter Rating given : 2
Song :  young and dumb
Enter Rating given : 3
Song :  apocalypse
Enter Rating given : 5
Song :  opera house
Enter Rating given : 6

Enter User  1 's rating for the following songs : 

Song :  love
Enter Rating given : 5
Song :  young and dumb
Enter Rating given : 4
Song :  apocalypse
Enter Rating given : 10
Song :  opera house
Enter Rating given : 3

Enter User  2 's rating for the following songs : 

Song :  love
Enter Rating given : 6
Song :  young and dumb
Enter Rating given : 5
Song :  apocalypse
Enter Rating given : 8
Song :  opera house
Enter Rating given : 9

Enter User  3 's rating for the following songs : 

Song :  love
Enter Rating given : 5
Song :  young and dumb
Enter Rating given : 4
Song :  apocalypse
Enter Rating given : 2
Song :  opera house
Enter Rating given : 1

Enter User  4 's rating for the follow

In [45]:
"""%matplotlib inline
from pylab import *

#Plot all the users
print("Matrix Dimensions for U")
print(U.shape)

for i in range(0, U.shape[0]):
    plot(U[i,0], U[i,1], marker = "*", label="user"+str(i))

for j in range(0, Vt.T.shape[0]):
    plot(Vt.T[j,0], Vt.T[j,1], marker = 'd', label="item"+str(j))    
    
legend(loc="upper right")
title('User vectors in the Latent semantic space')
ylim([-0.7, 0.7])
xlim([-0.7, 0])
show()"""

'%matplotlib inline\nfrom pylab import *\n\n#Plot all the users\nprint("Matrix Dimensions for U")\nprint(U.shape)\n\nfor i in range(0, U.shape[0]):\n    plot(U[i,0], U[i,1], marker = "*", label="user"+str(i))\n\nfor j in range(0, Vt.T.shape[0]):\n    plot(Vt.T[j,0], Vt.T[j,1], marker = \'d\', label="item"+str(j))    \n    \nlegend(loc="upper right")\ntitle(\'User vectors in the Latent semantic space\')\nylim([-0.7, 0.7])\nxlim([-0.7, 0])\nshow()'

In [53]:
##WORK IN PROGRESS - Word2Vec trainer for recommendations

In [58]:
import contractions as ctr
import itertools
import pandas
import pickle
import re
import tfidf
from tqdm import *
from gensim.models import Word2Vec

data = pandas.read_csv('./data/songdata.csv') # ~2 million sentences
N_SONGS = 5000

def preprocess(_text):
    try:
        _text = ctr.expand(_text) if _text.index("'") > -1 else _text
    except:
        pass
    return re.findall(r'\w+', _text.lower())

def text2sent(_lyrics):
    return list(filter(
        lambda x: len(x) > 0,
        [preprocess(s.rstrip()) for s in _lyrics.split('\n')]))

def text2doc(_lyrics):
    return list(itertools.chain.from_iterable(text2sent(_lyrics)))

def build_sentences(_data):
    dataset = []
    for song in _data:
        dataset += text2sent(song)
        tfidf.add(text2doc(song))
    return dataset

def doc2vec(_lyrics, _model):
    processed = text2doc(_lyrics)
    return sum(map(lambda z: tfidf.tfidf(z, processed) * _model[z], processed))

print "Building sentences..."
sentences = build_sentences(data['text'][:N_SONGS])
tfidf.init()

print "Training word2vec..."
model = Word2Vec(sentences, min_count=1, workers=4)
model.save('./data/model')

print "Building song vectors..."
vec = {data['song'][i]: doc2vec(data['text'][i], model) for i in tqdm(range(0, len(data['text'][:N_SONGS])))}
with open('./data/songvec.pickle', 'wb') as handle:
    pickle.dump(vec, handle, protocol=pickle.HIGHEST_PROTOCOL)

print "Done."

Building sentences...



  1%|▏         | 299/21177 [00:17<21:03, 16.53it/s]


  3%|▎         | 598/21177 [00:34<20:46, 16.51it/s]


  4%|▍         | 908/21177 [00:51<18:25, 18.33it/s]


  6%|▌         | 1222/21177 [01:08<17:52, 18.60it/s]


...


  0%|          | 0/5000 [00:00<?, ?it/s]/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

  0%|          | 25/5000 [00:00<00:20, 248.50it/s]

Building song vectors...



 92%|█████████▏| 4606/5000 [00:16<00:01, 206.84it/s]


100%|██████████| 5000/5000 [00:17<00:00, 280.82it/s]

Done.


In [61]:
#testing the data 

import numpy as np
import operator
import pickle
import tfidf
from gensim.models import Word2Vec

model = Word2Vec.load('./data/model')
with open('./data/songvec.pickle', 'rb') as handle:
    songs = pickle.load(handle)

def cos_sim(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

def nn(v, s):
    p = {song: cos_sim(v, vec) for song, vec in s.items()}
    return sorted(p.items(), key=operator.itemgetter(1), reverse=True)

def recommend_songs(song, top_n=5):
    return nn(songs[song], songs)[:top_n]

#change it to take input from the previous recommendation. Use the hybrid metric for song recommendation.x
_song = raw_input("Enter a recommendation for the song")

print(recommend_songs(_song))

Enter a recommendation for the songDancing Queen
[('Once Upon A Lifetime', nan), ('Honky Cat', nan), ('Nine Lives', nan), ('Old Songs', nan), ('Under The Weather', nan)]


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in float_scalars
  
